In [ ]:
#Loading dependencies
import matplotlib.pyplot as plt
import numpy as np
import pathlib
from sklearn.utils import shuffle
from tensorflow.keras.preprocessing.image import load_img
from keras.models import load_model

In [ ]:
#Sample image
image = load_img("./dataset/with_mask/0-with-mask.jpg", target_size = (120, 120))
plt.imshow(image)
plt.show()

In [ ]:
x_train = np.ndarray((1694, 150, 150, 3), dtype = "float32")
y = []
c = 0

In [ ]:
#Loading the images and corresponding labels
path  = path = pathlib.Path("./dataset/with_mask")
for i in path.iterdir():
    image = load_img(str(i), target_size = (150, 150))
    image = np.asarray(image)
    x_train[c] = image
    c = c + 1
    y.append(1)

In [ ]:
#Loading the images and corresponding labels
path  = path = pathlib.Path("./dataset/without_mask")
for i in path.iterdir():
    image = load_img(str(i), target_size = (150, 150)) 
    image = np.asarray(image)
    x_train[c] = image
    c = c + 1
    y.append(0)

In [ ]:
x_train.shape, len(y)

In [ ]:
y = np.array(y)
x_train.shape, y.shape

In [ ]:
X, y = shuffle(x_train, y, random_state = 17)

In [ ]:
#Normalising
X = X / 255.0

In [ ]:
X.shape, y.shape

In [ ]:
#Sample
plt.imshow(X[1].squeeze(), cmap = "gray"), y[1]

In [ ]:
#Dividing into train, validation and test sets
from sklearn.model_selection import train_test_split
X_train, X_validation, y_train, y_validation = train_test_split(X, y, test_size = 0.2, random_state = 7)
X_test, X_validation, y_test, y_validation = train_test_split(X_validation, y_validation, test_size = 0.5, random_state = 1)

In [ ]:
X_train.shape, X_validation.shape, X_test.shape, y_train.shape, y_validation.shape, y_test.shape

In [ ]:
#Converitng the labels into categorical form
import keras
number_cat = 2
y_train = keras.utils.to_categorical(y_train, number_cat)
y_validation = keras.utils.to_categorical(y_validation, number_cat)
y_test = keras.utils.to_categorical(y_test, number_cat)

In [ ]:
X_train.shape, X_validation.shape, X_test.shape, y_train.shape, y_validation.shape, y_test.shape

In [ ]:
#Sample
plt.imshow(X_train[1331].squeeze(), cmap = "gray"), y_train[0]

In [ ]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout
from keras.callbacks import TensorBoard
from keras.preprocessing.image import ImageDataGenerator

In [ ]:
model = Sequential()

model.add(Conv2D(filters = 32, kernel_size = (3, 3), activation = "relu", input_shape = (150, 150, 3)))
model.add(Conv2D(filters = 32, kernel_size = (3, 3), activation = "relu"))
model.add(MaxPooling2D(pool_size = (2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(filters = 64, kernel_size = (3, 3), activation = "relu"))
model.add(Conv2D(filters = 64, kernel_size = (3, 3), activation = "relu"))
model.add(MaxPooling2D(pool_size = (2, 2)))
model.add(Dropout(0.3))

model.add(Conv2D(filters = 128, kernel_size = (3, 3), activation = "relu"))
model.add(Conv2D(filters = 128, kernel_size = (3, 3), activation = "relu"))
model.add(Conv2D(filters = 128, kernel_size = (3, 3), activation = "relu"))
model.add(MaxPooling2D(pool_size = (2, 2)))
model.add(Dropout(0.33))

model.add(Flatten())

model.add(Dense(units = 256, activation = "relu"))

model.add(Dense(units = 512, activation = "relu"))
model.add(Dropout(0.33))

model.add(Dense(units = 1024, activation = "relu"))
model.add(Dropout(0.5))

model.add(Dense(units = 2, activation = "softmax"))

In [ ]:
model.summary()

In [ ]:
model.compile(loss = "categorical_crossentropy", optimizer= keras.optimizers.Adam(learning_rate=0.001), metrics = ["accuracy"])

In [ ]:
history = model.fit(X_train, y_train, epochs = 35, batch_size = 32 , validation_data = (X_validation, y_validation), verbose = 1)

In [ ]:
history.history.keys()

In [ ]:
#Analysing the performance

In [ ]:
plt.plot(history.history['loss'], "r", label = "training loss")
plt.plot(history.history['val_loss'], "b", label = "validation loss")
plt.xlabel("epochs")
plt.ylabel("losses")
plt.legend()
plt.show

In [ ]:
plt.plot(history.history['accuracy'], "r", label = "training loss")
plt.plot(history.history['val_accuracy'], "b", label = "validation loss")
plt.xlabel("epochs")
plt.ylabel("accuracy")
plt.legend()
plt.show

In [ ]:
model.save("Model.h5")

In [ ]:
evaluation = model.evaluate(X_test, y_test)
print("test accuracy: {}" .format(evaluation[1]))

In [ ]:
predicted_class = model.predict_classes(X_test)
predicted_class

In [ ]:
y_test

In [ ]:
y_test = y_test.argmax(1)

In [ ]:
y_test

In [ ]:
import seaborn as sns
from sklearn.metrics import confusion_matrix

cm = confusion_matrix(y_test, predicted_class)
cm
plt.figure(figsize  = (15, 15))
sns.heatmap(cm, annot = True)